In [1]:
!pip install huggingface_hub
!pip install transformers
!pip install accelerate  peft  bitsandbytes  trl
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

You will have to get approval to access the LLMA2 models on huggingface by filling the form on meta ai website.

In [2]:
from huggingface_hub import login

login()


In [3]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


In [4]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [5]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map={"": 0})


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from peft import prepare_model_for_kbit_training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


In [7]:
def print_trainable_parameters(model):
    """

    Prints the number of trainable parameters in the model.

    """

    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"trainable params: {trainable_params} | | all params: {all_param} | | trainable % : {100 * trainable_params / all_param}")


In [8]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


In [9]:
model = get_peft_model(model, config)

print_trainable_parameters(model)


trainable params: 4194304 | | all params: 3504607232 | | trainable % : 0.11967971650867153


In [10]:
from datasets import load_dataset
dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/timdettmers___json/timdettmers--openassistant-guanaco-6126c710748182cf/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


In [11]:
import transformers
from trl import SFTTrainer

# needed for llama tokenizer
tokenizer.pad_token = tokenizer.eos_token


trainer = SFTTrainer(
  model=model,
  train_dataset=dataset,
  dataset_text_field="text",
  args=transformers.TrainingArguments(
  per_device_train_batch_size=1,
  gradient_accumulation_steps=4,
  warmup_steps=2,
  max_steps=10,          #change accordingly
  learning_rate=2e-4,
  fp16=True,
  logging_steps=1,
  output_dir="outputs",
  optim="paged_adamw_8bit"
),
data_collator=transformers.DataCollatorForLanguageModeling(
    tokenizer, mlm=False),

)

model.config.use_cache=False  # silence the warnings. Please re-enable for inference!


Using pad_token, but it is not set yet.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

In [12]:
trainer.train()


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.538800
2,1.885500
3,2.269800
4,1.409600
5,2.073600
6,1.261000
7,1.908700
8,1.386600
9,2.098000
10,1.633900


TrainOutput(global_step=10, training_loss=1.8465556859970094, metrics={'train_runtime': 116.9114, 'train_samples_per_second': 0.342, 'train_steps_per_second': 0.086, 'total_flos': 409054641020928.0, 'train_loss': 1.8465556859970094, 'epoch': 0.0})

**Inference**





In [13]:
from transformers import pipeline
prompt = """### Human: Write a short story about two students trapped in a haunted house in Montana. ### Assistant:"""


pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            device_map="auto",
    )

sequences = pipe(
            prompt,
            max_length=1000,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
    )

for seq in sequences:
    print(seq['generated_text'])


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

### Human: Write a short story about two students trapped in a haunted house in Montana. ### Assistant: Sure, here is a short story about two students trapped in a haunted house in Montana. 

Two students, Emily and Ryan, were on their way to an overnight school trip in a small town in Montana. The bus ride there was long and boring, but they both looked forward to the adventure that awaited. They had planned to spend the night in an old abandoned house that was rumored to be haunted.
The trip to the house was through the woods and over uneven terrain. The sun had already set, and they were plunged into darkness. Emily, feeling anxious and scared, clung to Ryan. They heard strange noises all around them, and their flashlights flickered and went out. 
They found themselves trapped and alone in the dark. As the night continued, Ryan tried to reassure Emily with stories of other students who had also been there before. He even tried to tell ghost stories, hoping to make Emily laugh. Howev

In [16]:
from transformers import AutoTokenizer
import transformers
import torch

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)
sequences = pipeline(
    'I liked batman. Do you have any recommendations of other shows or movie I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    max_length=500,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

Result: I liked batman. Do you have any recommendations of other shows or movie I might like?
I have no doubt that if you are interested in the Dark Knight, that you will also enjoy other action packed super hero adventures. Here are three more suggestions that you might enjoy as well!
1. The Super Hero Squad Show.
The Super Hero Squadron Show is an animated television program that follows six superheroes from the Marvel Universe. Their goal is to save the world from evil villains and stop them from taking over the world. The show follows Ironman, Captain America, Thor, the Incredible Hulk, Spiderwoman and Wolverine on their super heroic adventures. The show is very much geared towards a young audience and will appeal to kids from 6-12 and parents alike!
2. Young Justice.
Young Justice is based on the DC Comics and follows the super heroic adventures of the teenage super heroes known as the Justice League. The show follows the adventures of Green Arrow, Robin, Super girl as well as oth